<a href="https://colab.research.google.com/github/mayureshpawashe/spark/blob/main/Spark_basic_day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import requests

csv_url = "https://gist.githubusercontent.com/kevin336/acbb2271e66c10a5b73aacf82ca82784/raw/e38afe62e088394d61ed30884dd50a6826eee0a8/employees.csv"

file_path = "/content/data.csv"

response = requests.get(csv_url)
with open(file_path, "wb") as f:
    f.write(response.content)

print("File downloaded successfully!")


File downloaded successfully!


In [21]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CSV_Operations").getOrCreate()
df = spark.read.csv(file_path, header=True, inferSchema=True)
df.show(10)


+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|  SH_CLERK|  2600|            - |       124|           50|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|  SH_CLERK|  2600|            - |       124|           50|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03|   AD_ASST|  4400|            - |       101|           10|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|    MK_MAN| 13000|            - |       100|           20|
|        202|       Pat|      Fay|    PFAY|603.123.6666|17-AUG-05|    MK_REP|  6000|            - |       201|           20|


In [23]:
df.printSchema()

root
 |-- EMPLOYEE_ID: integer (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- EMAIL: string (nullable = true)
 |-- PHONE_NUMBER: string (nullable = true)
 |-- HIRE_DATE: string (nullable = true)
 |-- JOB_ID: string (nullable = true)
 |-- SALARY: integer (nullable = true)
 |-- COMMISSION_PCT: string (nullable = true)
 |-- MANAGER_ID: string (nullable = true)
 |-- DEPARTMENT_ID: integer (nullable = true)



In [24]:
df.select("FIRST_NAME", "LAST_NAME", "SALARY").show()

+----------+---------+------+
|FIRST_NAME|LAST_NAME|SALARY|
+----------+---------+------+
|    Donald| OConnell|  2600|
|   Douglas|    Grant|  2600|
|  Jennifer|   Whalen|  4400|
|   Michael|Hartstein| 13000|
|       Pat|      Fay|  6000|
|     Susan|   Mavris|  6500|
|   Hermann|     Baer| 10000|
|   Shelley|  Higgins| 12008|
|   William|    Gietz|  8300|
|    Steven|     King| 24000|
|     Neena|  Kochhar| 17000|
|       Lex|  De Haan| 17000|
| Alexander|   Hunold|  9000|
|     Bruce|    Ernst|  6000|
|     David|   Austin|  4800|
|     Valli|Pataballa|  4800|
|     Diana|  Lorentz|  4200|
|     Nancy|Greenberg| 12008|
|    Daniel|   Faviet|  9000|
|      John|     Chen|  8200|
+----------+---------+------+
only showing top 20 rows



In [27]:
df.filter(df["SALARY"] > 15000).show()

+-----------+----------+---------+--------+------------+---------+-------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE| JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+-------+------+--------------+----------+-------------+
|        100|    Steven|     King|   SKING|515.123.4567|17-JUN-03|AD_PRES| 24000|            - |        - |           90|
|        101|     Neena|  Kochhar|NKOCHHAR|515.123.4568|21-SEP-05|  AD_VP| 17000|            - |       100|           90|
|        102|       Lex|  De Haan| LDEHAAN|515.123.4569|13-JAN-01|  AD_VP| 17000|            - |       100|           90|
+-----------+----------+---------+--------+------------+---------+-------+------+--------------+----------+-------------+



In [26]:
df.orderBy(df["SALARY"].desc()).show()

+-----------+-----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|EMPLOYEE_ID| FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+-----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|        100|     Steven|     King|   SKING|515.123.4567|17-JUN-03|   AD_PRES| 24000|            - |        - |           90|
|        101|      Neena|  Kochhar|NKOCHHAR|515.123.4568|21-SEP-05|     AD_VP| 17000|            - |       100|           90|
|        102|        Lex|  De Haan| LDEHAAN|515.123.4569|13-JAN-01|     AD_VP| 17000|            - |       100|           90|
|        201|    Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|    MK_MAN| 13000|            - |       100|           20|
|        205|    Shelley|  Higgins|SHIGGINS|515.123.8080|07-JUN-02|    AC_MGR| 12008|            - |       101|       

In [28]:
from pyspark.sql.functions import avg
df.select(avg("SALARY")).show()

+-----------+
|avg(SALARY)|
+-----------+
|    6182.32|
+-----------+



In [29]:
from pyspark.sql.functions import count

df.groupBy("DEPARTMENT_ID").agg(count("*").alias("Total_Employees")).show()

+-------------+---------------+
|DEPARTMENT_ID|Total_Employees|
+-------------+---------------+
|           20|              2|
|           40|              1|
|          100|              6|
|           10|              1|
|           50|             23|
|           70|              1|
|           90|              3|
|           60|              5|
|          110|              2|
|           30|              6|
+-------------+---------------+



##Finding Null (Missing) Values

In [30]:
from pyspark.sql.functions import col, count, when

df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-----------+----------+---------+-----+------------+---------+------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|EMAIL|PHONE_NUMBER|HIRE_DATE|JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+-----+------------+---------+------+------+--------------+----------+-------------+
|          0|         0|        0|    0|           0|        0|     0|     0|             0|         0|            0|
+-----------+----------+---------+-----+------------+---------+------+------+--------------+----------+-------------+



##Using SQL Queries on DataFrame

In [31]:
df.createOrReplaceTempView("employees")

result = spark.sql("SELECT FIRST_NAME, LAST_NAME, SALARY FROM employees WHERE SALARY > 15000")
result.show()

+----------+---------+------+
|FIRST_NAME|LAST_NAME|SALARY|
+----------+---------+------+
|    Steven|     King| 24000|
|     Neena|  Kochhar| 17000|
|       Lex|  De Haan| 17000|
+----------+---------+------+



In [33]:
import pandas as pd
pandas_df = df.toPandas()

# Display the Pandas DataFrame
pandas_df.head(10)


,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID
0,198,Donald,OConnell,DOCONNEL,650.507.9833,21-JUN-07,SH_CLERK,2600,-,124,50
1,199,Douglas,Grant,DGRANT,650.507.9844,13-JAN-08,SH_CLERK,2600,-,124,50
2,200,Jennifer,Whalen,JWHALEN,515.123.4444,17-SEP-03,AD_ASST,4400,-,101,10
3,201,Michael,Hartstein,MHARTSTE,515.123.5555,17-FEB-04,MK_MAN,13000,-,100,20
4,202,Pat,Fay,PFAY,603.123.6666,17-AUG-05,MK_REP,6000,-,201,20
5,203,Susan,Mavris,SMAVRIS,515.123.7777,07-JUN-02,HR_REP,6500,-,101,40
6,204,Hermann,Baer,HBAER,515.123.8888,07-JUN-02,PR_REP,10000,-,101,70
7,205,Shelley,Higgins,SHIGGINS,515.123.8080,07-JUN-02,AC_MGR,12008,-,101,110
8,206,William,Gietz,WGIETZ,515.123.8181,07-JUN-02,AC_ACCOUNT,8300,-,205,110
9,100,Steven,King,SKING,515.123.4567,17-JUN-03,AD_PRES,24000,-,-,90
